# Data Processing

This notebook is meant for all data cleaning and processing. Feel free to add or change sections.

Some Todos:
- `kernel_receivals`
    - Remove all rows where `rm_id`'s don't exist in `prediction_mapping`.
    - Remove all rows in `kernel_purchase_orders` with `quantity  <= 0`.
    - Remove `product_id` and `receival_status`.
    - Remove `batch_id`. It appears only from 2015 and onwards, and it has under 50% fill grade. This leads to poor data quality. In addition, it does not give predicitive information as all IDs are unique, and this can lead to overfitting in boosting-methods.
- `kernel_purchase_orders`
    - Remove `product_id`. There are multiple `rm_id`'s for each product_id, therefore it is useless here. We will know the rm_id after the dataset is joined with kernel_receivals based on purchase_order_id and/or purchase_order_item_no.
    - Remove `unit` and `unit_id`. Almost all entries have KG and the ones that do not are simply NaN and can be assumed to be KG as well.
    - Remove `status_id`. The status is duplicated in `status`.
    - Remove `product_version`. It is just set to 1 for all orders up until 2016 --> Poor data quality.
- `extended_materials`
    - Remove all rows where `rm_id`'s don't exist in `prediction_mapping`.
    - Remove `raw_material_alloy`, `product_id`.
    - There are a good deal of entries where `stock_location` is prefixed with DELETED followed by a date. It would be useful to check if there are differences in deliveries before and after that date. My hypothesis is that after it is deleted, there are no more deliveries for that rm_id. If this is a case, a boolean feature `is_deleted` would be powerful. Before inserting a DELETED=TRUE tag for an rm_id, check that there are no other listings of the same rm_id's that do not have DELETED in the stock_location variable. If there are other non-DELETED locations, then it may be that the stock location has simply moved but is still active. Be sure to also make a `deleted_date` feature as well. Then it is possible to create a `days_since_deleted = (forecast_date - deleted_date)` feature.
    - Remove `stock_location`, but only after making the `is_deleted` boolean and`date_deleted`.
- `extended_transportation`
    - Remove all rows where `rm_id`'s don't exist in `prediction_mapping`.
    - Remove `product_id`.
    - Remove `unit_status`. Almost all entries are 'Tranferred', a some are 'Accepted' and a couple are 'Pending'.
    - Remove `wood`, `ironbands`, `plastic`, `water`, `ice`, `other`, `chips`, `packaging`, `cardboard`. These have very few entries.



## Setup

### Imports

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from tabulate import tabulate
from typing import Dict

### Helper Functions

In [ ]:
def load_data(filename, folder="1_raw"):
    """
    Load data from a CSV file in a subfolder of the project's 'data' directory.
    This version is adjusted to work even if the notebook is run from a subfolder.

    Parameters
    ----------
    filename : str
        The name of the file to load, including the extension (e.g., "data.csv").
    folder : str, optional
        The subfolder within 'data' to load from. Defaults to "1_raw".
    """
    try:
        # Go up one level from the current working directory to find the project root
        PROJECT_ROOT = Path.cwd().parent

        file_path = PROJECT_ROOT / "data" / folder / filename

        df = pd.read_csv(file_path, sep=",")

        print(f"Data loaded successfully from {file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: The file was not found at {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return None


def save_data(df, filename, folder="2_interim"):
    """
    Save a dataframe to a CSV file in a subfolder of the project's 'data' directory.

    This function automatically creates the destination folder if it doesn't exist.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to save.
    filename : str
        The name for the output file, including the extension (e.g., "processed_orders.csv").
    folder : str, optional
        The subfolder within 'data' to save to. Defaults to "2_interim".
    """
    try:
        PROJECT_ROOT = Path.cwd().parent
        save_dir = PROJECT_ROOT / "data" / folder
        save_dir.mkdir(parents=True, exist_ok=True)

        # The full filename, including extension, is now expected
        file_path = save_dir / filename

        df.to_csv(file_path, sep=",", index=False)

        print(f"Data saved successfully to {file_path} ✅")

    except Exception as e:
        print(f"An error occurred while saving the file: {e}")


def generate_feature_presence_table(datasets: Dict[str, pd.DataFrame]):
    """
    Analyzes a dictionary of pandas DataFrames to determine the presence and
    fill grade of features (columns) across all of them. It then prints a
    formatted table summarizing this information.

    Args:
        datasets (Dict[str, pd.DataFrame]): A dictionary where keys are string
            names for the DataFrames and values are the pandas DataFrames
            themselves.
    """
    if not isinstance(datasets, dict) or not all(
        isinstance(df, pd.DataFrame) for df in datasets.values()
    ):
        print("Error: Input must be a dictionary of pandas DataFrames.")
        return

    # --- 1. Identify all unique features across all DataFrames ---
    all_features = sorted(
        list(set(feature for df in datasets.values() for feature in df.columns))
    )

    # --- 2. Build the presence and fill grade data for each DataFrame ---
    presence_data = []
    for name, df in datasets.items():
        row = {"DataFrame Name": name}
        for feature in all_features:
            if feature in df.columns:
                series = df[feature]
                total_count = len(series)

                if total_count > 0:
                    non_nan_count = series.count()  # .count() excludes NaNs
                    fill_grade = (non_nan_count / total_count) * 100
                    row[feature] = f"✅ {fill_grade:.1f}%"
                else:
                    # Handle empty DataFrames/Series
                    row[feature] = "✅ 100.0%"
            else:
                # Feature is not present in this DataFrame
                row[feature] = ""
        presence_data.append(row)

    # --- 3. Create, transpose, and format the DataFrame for display ---
    if not presence_data:
        print("No data to display.")
        return

    presence_df = pd.DataFrame(presence_data)
    presence_df = presence_df.set_index("DataFrame Name")

    # Transpose the DataFrame so features are rows and datasets are columns
    presence_df_transposed = presence_df.T
    presence_df_transposed.index.name = "Feature Name"

    # --- 4. Print the final table using tabulate ---
    print("--- Feature Presence & Fill Grade Across All DataFrames ---")
    print(
        tabulate(
            presence_df_transposed, headers="keys", tablefmt="grid", stralign="center"
        )
    )
    print("\n" * 2)  # Add some space after the table

### Data Loading

In [ ]:
# Kernel files
df_kpo = load_data("kernel_purchase_orders.csv")
df_kr = load_data("kernel_receivals.csv")

# Extended files
df_em = load_data("extended_materials.csv")
df_et = load_data("extended_transportation.csv")

## Filtering

In [ ]:
# Order datasets for iteration
datasets = {
    "kernel_receivals": df_kr,
    "kernel_purchase_orders": df_kpo,
    "extended_materials": df_em,
    "extended_transportation": df_et,
}
generate_feature_presence_table(datasets)

In [ ]:
# Load the mapping file to get the relevant rm_id's
df_map = load_data("prediction_mapping.csv")
relevant_rm_ids = df_map["rm_id"].unique()
print(f"Found {len(relevant_rm_ids)} unique rm_id's to focus on.")

In [ ]:
print("--- Processing kernel_receivals ---")

# Start with a copy to avoid modifying the original dataframe
df_kr_processed = df_kr.copy()

# Filter for relevant rm_id's
initial_rows = len(df_kr_processed)
df_kr_processed = df_kr_processed[df_kr_processed["rm_id"].isin(relevant_rm_ids)]
print(f"Filtered rm_ids: Kept {len(df_kr_processed)} rows out of {initial_rows}.")

# Remove specified columns
df_kr_processed = df_kr_processed.drop(
    columns=["product_id", "receival_status", "batch_id"]
)
print("Removed columns: product_id, receival_status, batch_id.")

# Ensure date_arrival is a datetime object for future joins and feature engineering
df_kr_processed["date_arrival"] = pd.to_datetime(df_kr_processed["date_arrival"])

print("kernel_receivals processing complete.")
df_kr_processed.info()

In [ ]:
print("\n--- Processing kernel_purchase_orders ---")

df_kpo_processed = df_kpo.copy()

# Remove orders with quantity <= 0
initial_rows = len(df_kpo_processed)
df_kpo_processed = df_kpo_processed[df_kpo_processed["quantity"] > 0]
print(
    f"Removed non-positive quantity orders: Kept {len(df_kpo_processed)} rows out of {initial_rows}."
)

# Remove specified columns
df_kpo_processed = df_kpo_processed.drop(
    columns=["product_id", "unit", "unit_id", "status_id", "product_version"]
)
print("Removed columns: product_id, unit, unit_id, status_id, product_version.")

print("kernel_purchase_orders processing complete.")
df_kpo_processed.info()

In [ ]:
print("\n--- Processing extended_materials ---")

df_em_processed = df_em.copy()

# Filter for relevant rm_id's
df_em_processed = df_em_processed[df_em_processed["rm_id"].isin(relevant_rm_ids)]

# --- Feature Engineering for DELETED stock locations ---

# Identify rows where stock_location indicates deletion
deleted_mask = df_em_processed["stock_location"].str.startswith("DELETED", na=False)
deleted_entries = df_em_processed[deleted_mask]

# Find rm_ids that ONLY have DELETED stock locations
# First, get all rm_ids that have at least one DELETED entry
rm_ids_with_deleted = deleted_entries["rm_id"].unique()

# Now, find which of those rm_ids ALSO have non-DELETED entries
active_locations_for_same_rm_id = df_em_processed[
    (df_em_processed["rm_id"].isin(rm_ids_with_deleted)) & (~deleted_mask)
]
rm_ids_with_mixed_status = active_locations_for_same_rm_id["rm_id"].unique()

# The truly deleted rm_ids are those that are in the deleted list but NOT in the mixed list
truly_deleted_rm_ids = set(rm_ids_with_deleted) - set(rm_ids_with_mixed_status)
print(f"Identified {len(truly_deleted_rm_ids)} rm_ids that are truly deleted.")

# Create the new boolean and date features
df_em_processed["is_deleted"] = df_em_processed["rm_id"].isin(truly_deleted_rm_ids)

# Extract the deleted date (this is a bit tricky, we'll use regex)
# This extracts the YYYY-MM-DD date from strings like "DELETED 2022-01-15 ..."
df_em_processed["deleted_date"] = df_em_processed["stock_location"].str.extract(
    r"(\d{4}-\d{2}-\d{2})"
)
df_em_processed["deleted_date"] = pd.to_datetime(df_em_processed["deleted_date"])

# Only keep the deleted_date for truly deleted materials
df_em_processed.loc[~df_em_processed["is_deleted"], "deleted_date"] = pd.NaT


# Remove specified columns
df_em_processed = df_em_processed.drop(
    columns=["raw_material_alloy", "product_id", "stock_location"]
)
print("Removed columns: raw_material_alloy, product_id, stock_location.")

# We might have multiple rows per rm_id now, so let's keep the most relevant one.
# We can group by rm_id and take the first entry, as the deleted flags are now consistent per rm_id.
df_em_processed = df_em_processed.groupby("rm_id").first().reset_index()


print("extended_materials processing complete.")
df_em_processed.info()

In [ ]:
print("\n--- Processing extended_transportation ---")

df_et_processed = df_et.copy()

# Filter for relevant rm_id's
df_et_processed = df_et_processed[df_et_processed["rm_id"].isin(relevant_rm_ids)]

# Remove specified columns
columns_to_drop = [
    "product_id",
    "unit_status",
    "wood",
    "ironbands",
    "plastic",
    "water",
    "ice",
    "other",
    "chips",
    "packaging",
    "cardboard",
]
df_et_processed = df_et_processed.drop(columns=columns_to_drop)
print(f"Removed columns: {', '.join(columns_to_drop)}.")

print("extended_transportation processing complete.")
df_et_processed.info()

In [ ]:
print("\n--- Joining into master_df ---")

# Use a composite key for joining purchase orders and transportation data
# to handle cases where a single order has multiple items or receivals.
key_cols = ["purchase_order_id", "purchase_order_item_no"]

# Start with receivals as the base (the "fact" table)
master_df = pd.merge(df_kr_processed, df_kpo_processed, on=key_cols, how="left")
print(f"Joined receivals and purchase orders. Shape: {master_df.shape}")

# Merge materials data (on rm_id)
master_df = pd.merge(master_df, df_em_processed, on="rm_id", how="left")
print(f"Joined materials data. Shape: {master_df.shape}")

# Merge transportation data
# Note: Transportation data might have more keys, e.g., receival_item_no.
# For simplicity, we'll join on the primary keys. You may need to refine this join
# if you find it creates duplicates.
transport_key_cols = key_cols + ["rm_id"]
master_df = pd.merge(
    master_df,
    df_et_processed,
    on=transport_key_cols,
    how="left",
    suffixes=("", "_transport"),
)
print(f"Joined transportation data. Shape: {master_df.shape}")

# --- Final Sanity Checks ---
master_df.info()

# --- FORMAT `master_df` FOR LIGHTGBM ---

def _parse_datetime_utc_series(s: pd.Series) -> pd.Series:
    # Normalize any " +00:00" artifacts to UTC datetimes
    s = s.astype("string").str.replace(r"\s\+", "+", regex=True)
    return pd.to_datetime(s, utc=True, errors="coerce")

def _to_int_if_whole_or_float_series(s: pd.Series) -> pd.Series:
    # Cast to Int32 if all non-null values are whole numbers, otherwise float32
    x = pd.to_numeric(s, errors="coerce")
    if (x.dropna() % 1 == 0).all():
        return x.astype("Int32")
    else:
        return x.astype("float32")

# Column groups
date_cols = ["date_arrival", "delivery_date", "created_date_time", "modified_date_time", "deleted_date"]
id_cols   = ["rm_id", "purchase_order_id", "purchase_order_item_no", "receival_item_no", "receival_item_no_transport"]  # batch_id removed
bool_cols = ["is_deleted"]
maybe_int_or_float = [
    "net_weight", "quantity", "product_version", "vehicle_start_weight",
    "vehicle_end_weight", "gross_weight", "tare_weight", "net_weight_transport"
]
# Treat these as categorical features (even if they look numeric)
cat_cols  = ["supplier_id", "status", "raw_material_format_type", "transporter_name", "vehicle_no"]

_df = master_df.copy()

# Drop batch_id entirely if present
if "batch_id" in _df.columns:
    _df = _df.drop(columns=["batch_id"])

# Keep only columns that exist
date_cols = [c for c in date_cols if c in _df.columns]
id_cols   = [c for c in id_cols   if c in _df.columns]
bool_cols = [c for c in bool_cols if c in _df.columns]
maybe_int_or_float = [c for c in maybe_int_or_float if c in _df.columns]
cat_cols  = [c for c in cat_cols  if c in _df.columns]

# 1) Datetimes -> UTC
for c in date_cols:
    _df[c] = _parse_datetime_utc_series(_df[c])

# 2) Booleans
for c in bool_cols:
    _df[c] = (
        _df[c].astype("string").str.strip().str.lower()
        .map({"true": True, "false": False, "1": True, "0": False})
        .astype("boolean")
    )

# 3) Numeric smart cast
for c in maybe_int_or_float:
    _df[c] = _to_int_if_whole_or_float_series(_df[c])

# 3b) Force 'quantity' to nullable Int64
if "quantity" in _df.columns:
    q = pd.to_numeric(_df["quantity"], errors="coerce")
    _df["quantity"] = q.round().astype("Int64")

# 4) IDs -> nullable Int64
for c in id_cols:
    _df[c] = pd.to_numeric(_df[c], errors="coerce").astype("Int64")

# 5) Categoricals -> int codes. store mapping (unknown -> -1)
cat_maps = {}
for c in cat_cols:
    _df[c] = _df[c].astype("string").str.strip().replace({"": pd.NA}).astype("category")
    categories = list(_df[c].cat.categories.astype("string"))
    mapping = {cat: idx for idx, cat in enumerate(categories)}
    codes = _df[c].cat.codes.astype("Int32")  # pandas uses -1 for NaN
    _df[c] = codes
    cat_maps[c] = {"mapping": mapping, "unknown_value": -1}

# 6) Report constant columns (excluding IDs)
constant_cols = [c for c in _df.columns if _df[c].nunique(dropna=True) <= 1 and c not in id_cols]
if len(constant_cols):
    print("Constant columns:", constant_cols)

# 7) Persist artifacts (dtype report + category maps)
art_dir = Path("./artifacts")
art_dir.mkdir(parents=True, exist_ok=True)

with open(art_dir / "master_data_dtype_report.json", "w", encoding="utf-8") as f:
    json.dump({c: str(_df[c].dtype) for c in _df.columns}, f, indent=2, ensure_ascii=False)

with open(art_dir / "category_maps.json", "w", encoding="utf-8") as f:
    json.dump(cat_maps, f, indent=2, ensure_ascii=False)

# --- Row ordering ---

def _coerce_rm_id_for_sort(s):
    # Force numeric
    return pd.to_numeric(s, errors="coerce")

def _coerce_dt_for_sort(s):
    # Parse with UTC
    return pd.to_datetime(s, utc=True, errors="coerce")

# Choose delivery date column
_dt_col = None
for candidate in ["delivery_date", "date_delivery", "agreed_delivery_date"]:
    if candidate in _df.columns:
        _dt_col = candidate
        break
if _dt_col is None:
    raise KeyError("Expected a delivery date column was not found.")

# Build temporary sort keys to avoid dtype/categorical issues
_rm_key = _coerce_rm_id_for_sort(_df["rm_id"])
_dt_key = _coerce_dt_for_sort(_df[_dt_col])

# Stable sort: rm_id ascending, delivery_date descending
_df_sorted = (
    _df.assign(_k_rm=_rm_key, _k_dt=_dt_key)
       .sort_values(["_k_rm", "_k_dt"], ascending=[True, False],
                    kind="mergesort", na_position="last")
       .drop(columns=["_k_rm", "_k_dt"])
       .reset_index(drop=True)
)

# Finalize for downstream code
master_df = _df_sorted
try:
    master_data = master_df  # if other parts expect this name
except Exception:
    pass

print("master_df formatted. Dtypes:\n", master_df.dtypes)

# --- Save the Processed Data ---
# Parquet preserves dtypes better; CSV is fine if required by your pipeline
# save_data(master_df, "master_data.parquet", folder="2_interim")
save_data(master_df, "master_data.csv", folder="2_interim")

In [ ]:
# Order datasets for iteration
df_m = load_data("master_data.csv", "2_interim")

datasets = {
    "master_data": df_m,
}
generate_feature_presence_table(datasets)